<a href="https://colab.research.google.com/github/eghib22/Store-Sales-Forecasting/blob/main/model_experiment_N_Beats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle
! pip install wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! mkdir ~/.kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ekaterineghibradze","key":"b1414052fbae86987efff2083c8dcbd1"}'}

In [5]:
!mv "kaggle.json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!ls -l ~/.kaggle/

total 4
-rw------- 1 root root 74 Jul  6 15:15 kaggle.json


In [7]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 840MB/s]


In [8]:
! unzip walmart-recruiting-store-sales-forecasting

Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [9]:
!unzip '*.csv.zip'

Archive:  features.csv.zip
  inflating: features.csv            

Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  test.csv.zip
  inflating: test.csv                

4 archives were successfully processed.


In [10]:
!unzip '*.csv.zip'

Archive:  features.csv.zip
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv            

Archive:  sampleSubmission.csv.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv    

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               

Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                

4 archives were successfully processed.


In [11]:
import pandas as pd

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')
sample_submission = pd.read_csv('sampleSubmission.csv')

# Explore the data
print("Train shape:", train.shape)
print(train.head())
print(train.columns)

print("\nTest shape:", test.shape)
print(test.head())
print(test.columns)

print("\nFeatures shape:", features.shape)
print(features.head())
print(features.columns)

print("\nStores shape:", stores.shape)
print(stores.head())
print(stores.columns)

print("\nSample Submission shape:", sample_submission.shape)
print(sample_submission.head())
print(sample_submission.columns)

Train shape: (421570, 5)
   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False
Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday'], dtype='object')

Test shape: (115064, 4)
   Store  Dept        Date  IsHoliday
0      1     1  2012-11-02      False
1      1     1  2012-11-09      False
2      1     1  2012-11-16      False
3      1     1  2012-11-23       True
4      1     1  2012-11-30      False
Index(['Store', 'Dept', 'Date', 'IsHoliday'], dtype='object')

Features shape: (8190, 12)
   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        3

In [12]:
# Merge train and test with features and stores
train_merged = pd.merge(train, features, on=['Store', 'Date'], how='left')
train_merged = pd.merge(train_merged, stores, on='Store', how='left')

test_merged = pd.merge(test, features, on=['Store', 'Date'], how='left')
test_merged = pd.merge(test_merged, stores, on='Store', how='left')

# Convert Date to datetime
train_merged['Date'] = pd.to_datetime(train_merged['Date'])


In [13]:
# Split
train_data = train_merged[train_merged['Date'] < '2012-01-01']
val_data = train_merged[(train_merged['Date'] >= '2012-01-01') & (train_merged['Date'] < '2012-07-01')]
test_data = train_merged[train_merged['Date'] >= '2012-07-01']

print("Train:", train_data.shape)
print("Validation:", val_data.shape)
print("Test (local):", test_data.shape)

Train: (294132, 17)
Validation: (77110, 17)
Test (local): (50328, 17)


In [14]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eghib22 (eghib22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [15]:
def preprocess(df):
    df = df.copy()

    # Map Type A/B/C to 0/1/2
    type_map = {'A': 0, 'B': 1, 'C': 2}
    df['Type'] = df['Type'].map(type_map)

    # Ensure IsHoliday is int (True/False → 1/0)
    if 'IsHoliday_x' in df.columns:
        df['IsHoliday'] = df['IsHoliday_x'].astype(int)
        df = df.drop(columns=['IsHoliday_x', 'IsHoliday_y'])
    elif 'IsHoliday' in df.columns:
        df['IsHoliday'] = df['IsHoliday'].astype(int)

    # Fill NaNs in MarkDown columns with 0
    markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
    for col in markdown_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    # Extract date features
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
    df['Day'] = df['Date'].dt.day

    return df


In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import warnings
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# WandB configuration
wandb_config = {
    "epochs": 50,
    "batch_size": 32,
    "learning_rate": 0.001,
    "lookback": 12,
    "horizon": 1,
    "num_blocks": 3,
    "units": 256,
    "model_type": "N-BEATS",
    "dataset": "Walmart Sales",
    "optimizer": "Adam",
    "loss": "MSE"
}

class NBeatsBlock(layers.Layer):
    """N-BEATS block implementation"""

    def __init__(self, units, thetas_dim, horizon, backcast_length, share_weights=False, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.thetas_dim = thetas_dim
        self.horizon = horizon
        self.backcast_length = backcast_length
        self.share_weights = share_weights

        # Fully connected layers
        self.fc1 = layers.Dense(units, activation='relu')
        self.fc2 = layers.Dense(units, activation='relu')
        self.fc3 = layers.Dense(units, activation='relu')
        self.fc4 = layers.Dense(units, activation='relu')

        # Theta layers
        self.theta_f = layers.Dense(thetas_dim, activation='linear')
        self.theta_b = layers.Dense(thetas_dim, activation='linear')

        # Basis functions (simplified - using linear basis)
        self.basis_f = layers.Dense(horizon, activation='linear')
        self.basis_b = layers.Dense(backcast_length, activation='linear')

    def call(self, inputs):
        # Forward pass through FC layers
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)

        # Generate thetas
        theta_f = self.theta_f(x)
        theta_b = self.theta_b(x)

        # Generate forecast and backcast
        forecast = self.basis_f(theta_f)
        backcast = self.basis_b(theta_b)

        return forecast, backcast

    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units,
            'thetas_dim': self.thetas_dim,
            'horizon': self.horizon,
            'backcast_length': self.backcast_length,
            'share_weights': self.share_weights
        })
        return config

def preprocess(df):
    """Preprocess the data"""
    # Encode categorical type
    type_map = {'A': 0, 'B': 1, 'C': 2}
    df = df.copy()  # Avoid SettingWithCopyWarning

    # Handle Type column
    if 'Type' in df.columns:
        df['Type'] = df['Type'].map(type_map)
        df['Type'] = df['Type'].fillna(0)  # Fill any NaN with 0

    # Handle IsHoliday columns
    if 'IsHoliday_x' in df.columns:
        df['IsHoliday'] = df['IsHoliday_x'].astype(int)
        df = df.drop(columns=['IsHoliday_x', 'IsHoliday_y'])
    elif 'IsHoliday' in df.columns:
        df['IsHoliday'] = df['IsHoliday'].astype(int)

    # Date features (before dropping Date)
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
        df['Year'] = df['Date'].dt.year.astype(int)
        df['Month'] = df['Date'].dt.month.astype(int)
        df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
        df['Day'] = df['Date'].dt.day.astype(int)
        df['DayOfWeek'] = df['Date'].dt.dayofweek.astype(int)
        df['DayOfYear'] = df['Date'].dt.dayofyear.astype(int)

        # Drop Date column after extracting features
        df = df.drop(columns=['Date'])

    # Fill MarkDown NaNs
    markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
    for col in markdown_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0.0)

    # Ensure all numeric columns are proper numeric types
    numeric_cols = ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

def create_sequences(data, lookback, horizon, target_col='Weekly_Sales'):
    """Create sequences for time series forecasting"""
    sequences = []
    targets = []

    # Ensure we have the right columns
    if target_col not in data.columns:
        raise ValueError(f"Target column '{target_col}' not found in data")

    # Get feature columns (excluding target)
    feature_cols = [col for col in data.columns if col != target_col]

    print(f"Creating sequences with {len(feature_cols)} features: {feature_cols}")

    # Group by Store and Dept to maintain temporal order
    for (store, dept), group in data.groupby(['Store', 'Dept']):
        # Reset index to ensure continuous indexing
        group = group.reset_index(drop=True)

        if len(group) < lookback + horizon:
            continue

        # Create sequences for this store-dept combination
        for i in range(len(group) - lookback - horizon + 1):
            # Input sequence (features only)
            seq_features = group.iloc[i:i+lookback][feature_cols].values

            # Target sequence
            target_seq = group.iloc[i+lookback:i+lookback+horizon][target_col].values

            # Ensure correct shape
            if seq_features.shape[0] == lookback and len(target_seq) == horizon:
                sequences.append(seq_features)
                targets.append(target_seq)

    sequences = np.array(sequences, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    print(f"Created {len(sequences)} sequences with shape: {sequences.shape}")
    print(f"Target shape: {targets.shape}")

    return sequences, targets

def create_walmart_nbeats_model(input_shape, horizon=1, num_blocks=3, units=256):
    """Create simplified N-BEATS inspired model for Walmart sales forecasting"""

    # Input layer
    inputs = layers.Input(shape=input_shape, name='input_layer')

    # Flatten the input for N-BEATS
    flattened = layers.Flatten(name='flatten')(inputs)

    # Add some preprocessing layers
    x = layers.Dense(512, activation='relu', name='dense_preprocess_1')(flattened)
    x = layers.Dropout(0.2, name='dropout_1')(x)
    x = layers.Dense(256, activation='relu', name='dense_preprocess_2')(x)
    x = layers.Dropout(0.2, name='dropout_2')(x)

    # Simplified N-BEATS inspired blocks
    forecasts = []
    residual = x

    for i in range(num_blocks):
        # FC layers for this block
        block_out = layers.Dense(units, activation='relu', name=f'block_{i}_dense_1')(residual)
        block_out = layers.Dense(units, activation='relu', name=f'block_{i}_dense_2')(block_out)
        block_out = layers.Dense(units, activation='relu', name=f'block_{i}_dense_3')(block_out)

        # Forecast and backcast
        forecast = layers.Dense(horizon, activation='linear', name=f'forecast_{i}')(block_out)
        backcast = layers.Dense(units, activation='linear', name=f'backcast_{i}')(block_out)

        forecasts.append(forecast)

        # Residual connection (make sure dimensions match)
        if residual.shape[-1] != backcast.shape[-1]:
            residual = layers.Dense(units, name=f'residual_proj_{i}')(residual)

        residual = layers.Subtract(name=f'residual_subtract_{i}')([residual, backcast])

    # Sum all forecasts
    if len(forecasts) > 1:
        output = layers.Add(name='sum_forecasts')(forecasts)
    else:
        output = forecasts[0]

    # Final output layer
    output = layers.Dense(horizon, activation='linear', name='final_output')(output)

    model = keras.Model(inputs=inputs, outputs=output, name='walmart_nbeats')
    return model

def calculate_wmae(y_true, y_pred, weights=None):
    """Calculate Weighted Mean Absolute Error"""
    if weights is None:
        weights = np.ones_like(y_true)

    # Ensure arrays are flattened
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    weights = weights.flatten()

    # Calculate weighted MAE
    mae = np.abs(y_true - y_pred)
    wmae = np.sum(weights * mae) / np.sum(weights)

    return wmae

def prepare_walmart_data(train_data, val_data, test_data, lookback=12, horizon=1):
    """Prepare data for N-BEATS model"""

    # Preprocess all datasets
    train_processed = preprocess(train_data.copy())
    val_processed = preprocess(val_data.copy())
    test_processed = preprocess(test_data.copy())

    # Get feature columns (excluding target and any remaining non-numeric columns)
    feature_cols = [col for col in train_processed.columns
                   if col not in ['Weekly_Sales', 'Date'] and
                   train_processed[col].dtype in ['int64', 'float64', 'int32', 'float32']]

    print(f"Feature columns: {feature_cols}")
    print(f"Number of features: {len(feature_cols)}")

    # Debug: Check data types
    print("Data types in train_processed:")
    print(train_processed[feature_cols].dtypes)

    # Scale features (excluding target)
    scaler_features = StandardScaler()
    scaler_target = StandardScaler()

    # Fit scalers on training data
    train_features = train_processed[feature_cols].astype(np.float32)
    train_target = train_processed['Weekly_Sales'].values.reshape(-1, 1).astype(np.float32)

    scaler_features.fit(train_features)
    scaler_target.fit(train_target)

    # Transform all datasets
    train_processed[feature_cols] = scaler_features.transform(train_features)
    train_processed['Weekly_Sales'] = scaler_target.transform(train_target).flatten()

    val_features = val_processed[feature_cols].astype(np.float32)
    val_target = val_processed['Weekly_Sales'].values.reshape(-1, 1).astype(np.float32)
    val_processed[feature_cols] = scaler_features.transform(val_features)
    val_processed['Weekly_Sales'] = scaler_target.transform(val_target).flatten()

    test_features = test_processed[feature_cols].astype(np.float32)
    test_target = test_processed['Weekly_Sales'].values.reshape(-1, 1).astype(np.float32)
    test_processed[feature_cols] = scaler_features.transform(test_features)
    test_processed['Weekly_Sales'] = scaler_target.transform(test_target).flatten()

    # Create sequences
    X_train, y_train = create_sequences(train_processed, lookback, horizon)
    X_val, y_val = create_sequences(val_processed, lookback, horizon)
    X_test, y_test = create_sequences(test_processed, lookback, horizon)

    # Ensure correct data types
    X_train = X_train.astype(np.float32)
    y_train = y_train.astype(np.float32)
    X_val = X_val.astype(np.float32)
    y_val = y_val.astype(np.float32)
    X_test = X_test.astype(np.float32)
    y_test = y_test.astype(np.float32)

    print(f"Training sequences: {X_train.shape}, {y_train.shape}")
    print(f"Validation sequences: {X_val.shape}, {y_val.shape}")
    print(f"Test sequences: {X_test.shape}, {y_test.shape}")
    print(f"Data types - X_train: {X_train.dtype}, y_train: {y_train.dtype}")

    return (X_train, y_train, X_val, y_val, X_test, y_test,
            scaler_features, scaler_target)

def train_nbeats_model(X_train, y_train, X_val, y_val,
                      epochs=100, batch_size=64, learning_rate=0.001):
    """Train N-BEATS model with WandB logging"""

    # Create model
    model = create_walmart_nbeats_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        horizon=y_train.shape[1],
        num_blocks=wandb_config["num_blocks"],
        units=wandb_config["units"]
    )

    # Print model summary
    print("Model Summary:")
    model.summary()

    # Log model architecture to WandB
    wandb.log({"model_parameters": model.count_params()})

    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )

    # Callbacks with WandB integration
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        ),
        WandbMetricsLogger(log_freq="epoch"),  # New WandB metrics logger
        WandbModelCheckpoint(
            filepath="best_model.h5",
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=False
        )
    ]

    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )

    return model, history

def evaluate_model(model, X_test, y_test, scaler_target):
    """Evaluate model and calculate WMAE with WandB logging"""

    # Make predictions
    y_pred_scaled = model.predict(X_test)

    # Inverse transform predictions and targets
    y_pred = scaler_target.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    y_true = scaler_target.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    wmae = calculate_wmae(y_true, y_pred)

    # Calculate WMAE with holiday weights (higher weight for holiday periods)
    holiday_weights = np.where(y_true > np.median(y_true), 5, 1)  # Higher weight for higher sales
    wmae_weighted = calculate_wmae(y_true, y_pred, holiday_weights)

    # Calculate additional metrics
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # Log all metrics to WandB
    wandb.log({
        "test_mae": mae,
        "test_wmae": wmae,
        "test_wmae_weighted": wmae_weighted,
        "test_mse": mse,
        "test_rmse": rmse,
        "test_mape": mape
    })

    # Create and log prediction plots
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Scatter plot
    sample_idx = np.random.choice(len(y_true), min(1000, len(y_true)), replace=False)
    axes[0].scatter(y_true[sample_idx], y_pred[sample_idx], alpha=0.6)
    axes[0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    axes[0].set_xlabel('Actual Sales')
    axes[0].set_ylabel('Predicted Sales')
    axes[0].set_title('Predictions vs Actual')

    # Time series plot
    sample_range = slice(0, min(100, len(y_true)))
    axes[1].plot(y_true[sample_range], label='Actual', marker='o', markersize=3)
    axes[1].plot(y_pred[sample_range], label='Predicted', marker='s', markersize=3)
    axes[1].set_xlabel('Time Steps')
    axes[1].set_ylabel('Sales')
    axes[1].set_title('Time Series Sample')
    axes[1].legend()

    plt.tight_layout()

    # Log plots to WandB
    wandb.log({"predictions_plot": wandb.Image(fig)})
    plt.close(fig)

    # Log prediction distribution
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    ax.hist(y_true, bins=50, alpha=0.7, label='Actual', density=True)
    ax.hist(y_pred, bins=50, alpha=0.7, label='Predicted', density=True)
    ax.set_xlabel('Sales Values')
    ax.set_ylabel('Density')
    ax.set_title('Distribution of Actual vs Predicted Sales')
    ax.legend()

    wandb.log({"distribution_plot": wandb.Image(fig)})
    plt.close(fig)

    # Print metrics
    print(f"Test MAE: {mae:.2f}")
    print(f"Test WMAE: {wmae:.2f}")
    print(f"Test WMAE (Holiday weighted): {wmae_weighted:.2f}")
    print(f"Test RMSE: {rmse:.2f}")
    print(f"Test MAPE: {mape:.2f}%")

    return y_pred, y_true, mae, wmae, wmae_weighted

def plot_predictions(y_true, y_pred, n_samples=100):
    """Plot predictions vs actual values"""
    plt.figure(figsize=(15, 5))

    # Plot sample predictions
    plt.subplot(1, 2, 1)
    sample_idx = np.random.choice(len(y_true), min(n_samples, len(y_true)), replace=False)
    plt.scatter(y_true[sample_idx], y_pred[sample_idx], alpha=0.6)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    plt.xlabel('Actual Sales')
    plt.ylabel('Predicted Sales')
    plt.title('Predictions vs Actual (Sample)')

    # Plot time series sample
    plt.subplot(1, 2, 2)
    sample_range = slice(0, min(50, len(y_true)))
    plt.plot(y_true[sample_range], label='Actual', marker='o')
    plt.plot(y_pred[sample_range], label='Predicted', marker='s')
    plt.xlabel('Time Steps')
    plt.ylabel('Sales')
    plt.title('Time Series Sample')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Main execution
if __name__ == "__main__":
    # Initialize WandB
    wandb.init(
        project="Store-Sales-Forecasting",
        entity="agasi22-free-university-of-tbilisi-" ,
        name="n-beats-training-run",
        config=wandb_config,
        tags=["nbeats", "time-series", "walmart", "sales-forecasting"]
    )

    # Update config with wandb config
    config = wandb.config

    print("🚀 Starting Walmart Sales Forecasting with N-BEATS")
    print(f"📊 WandB Project: {wandb.run.project}")
    print(f"🔧 Run Name: {wandb.run.name}")

    # Assuming you have already loaded and preprocessed your data
    # train_data, val_data, test_data should be available

    print("\n📈 Preparing data for N-BEATS model...")
    (X_train, y_train, X_val, y_val, X_test, y_test,
     scaler_features, scaler_target) = prepare_walmart_data(
        train_data, val_data, test_data,
        lookback=config.lookback, horizon=config.horizon
    )

    # Log data statistics
    wandb.log({
        "train_sequences": len(X_train),
        "val_sequences": len(X_val),
        "test_sequences": len(X_test),
        "feature_dimensions": X_train.shape[2],
        "lookback_window": config.lookback,
        "forecast_horizon": config.horizon
    })

    print("\n🤖 Training N-BEATS model...")
    model, history = train_nbeats_model(
        X_train, y_train, X_val, y_val,
        epochs=config.epochs,
        batch_size=config.batch_size,
        learning_rate=config.learning_rate
    )

    print("\n📊 Evaluating model...")
    y_pred, y_true, mae, wmae, wmae_weighted = evaluate_model(
        model, X_test, y_test, scaler_target
    )

    print("\n📈 Creating training history plots...")
    # Plot training history and log to WandB
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    axes[0].plot(history.history['loss'], label='Training Loss')
    axes[0].plot(history.history['val_loss'], label='Validation Loss')
    axes[0].set_title('Model Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()

    axes[1].plot(history.history['mae'], label='Training MAE')
    axes[1].plot(history.history['val_mae'], label='Validation MAE')
    axes[1].set_title('Model MAE')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('MAE')
    axes[1].legend()

    plt.tight_layout()
    wandb.log({"training_history": wandb.Image(fig)})
    plt.close(fig)

    # Log final summary
    wandb.log({
        "final_train_loss": history.history['loss'][-1],
        "final_val_loss": history.history['val_loss'][-1],
        "final_train_mae": history.history['mae'][-1],
        "final_val_mae": history.history['val_mae'][-1],
        "best_val_loss": min(history.history['val_loss']),
        "training_epochs": len(history.history['loss'])
    })

    print(f"\n✅ Training completed successfully!")
    print(f"📊 Final Results:")
    print(f"   • Test MAE: {mae:.2f}")
    print(f"   • Test WMAE: {wmae:.2f}")
    print(f"   • Test WMAE (Holiday weighted): {wmae_weighted:.2f}")
    print(f"🔗 View results at: {wandb.run.url}")

    # Save model artifact to WandB
    model_artifact = wandb.Artifact("walmart-nbeats-model", type="model")
    model.save("walmart_nbeats_model.h5")
    model_artifact.add_file("walmart_nbeats_model.h5")
    wandb.log_artifact(model_artifact)

    # Finish WandB run
    wandb.finish()
    print("🎉 WandB logging completed!")

🚀 Starting Walmart Sales Forecasting with N-BEATS
📊 WandB Project: Store-Sales-Forecasting
🔧 Run Name: n-beats-training-run

📈 Preparing data for N-BEATS model...
Feature columns: ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'IsHoliday', 'Year', 'Month', 'Week', 'Day', 'DayOfWeek', 'DayOfYear']
Number of features: 20
Data types in train_processed:
Store             int64
Dept              int64
Temperature     float64
Fuel_Price      float64
MarkDown1       float64
MarkDown2       float64
MarkDown3       float64
MarkDown4       float64
MarkDown5       float64
CPI             float64
Unemployment    float64
Type              int64
Size              int64
IsHoliday         int64
Year              int64
Month             int64
Week              int64
Day               int64
DayOfWeek         int64
DayOfYear         int64
dtype: object
Creating sequences with 20 features: ['Store', 'De

Model: "walmart_nbeats"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 12, 20)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 240)       │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_preprocess_1  │ (None, 512)       │    123,392 │ flatten[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 512)       │          0 │ dense_preprocess… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_preprocess_2  │ (None, 256)       │    131,328 │ dropout_1[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ dense_preprocess… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_dense_1     │ (None, 256)       │     65,792 │ dropout_2[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_dense_2     │ (None, 256)       │     65,792 │ block_0_dense_1[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_0_dense_3     │ (None, 256)       │     65,792 │ block_0_dense_2[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backcast_0 (Dense)  │ (None, 256)       │     65,792 │ block_0_dense_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residual_subtract_0 │ (None, 256)       │          0 │ dropout_2[0][0],  │
│ (Subtract)          │                   │            │ backcast_0[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_dense_1     │ (None, 256)       │     65,792 │ residual_subtrac… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_dense_2     │ (None, 256)       │     65,792 │ block_1_dense_1[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_dense_3     │ (None, 256)       │     65,792 │ block_1_dense_2[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backcast_1 (Dense)  │ (None, 256)       │     65,792 │ block_1_dense_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residual_subtract_1 │ (None, 256)       │          0 │ residual_subtrac… │
│ (Subtract)          │                   │            │ backcast_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_2_dense_1     │ (None, 256)       │     65,792 │ residual_subtrac… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_2_dense_2     │ (None, 256)       │     65,792 │ block_2_dense_1[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 979,205 (3.74 MB)

 Trainable params: 979,205 (3.74 MB)

...

8002/8002 ━━━━━━━━━━━━━━━━━━━━ 196s 17ms/step - loss: 0.6302 - mae: 0.4859 - val_loss: 1.5593 - val_mae: 1.0527 - learning_rate: 0.0010
Epoch 6/50
8002/8002 ━━━━━━━━━━━━━━━━━━━━ 138s 17ms/step - loss: 0.5872 - mae: 0.4659 - val_loss: 1.6296 - val_mae: 1.0398 - learning_rate: 0.0010
Epoch 7/50
8002/8002 ━━━━━━━━━━━━━━━━━━━━ 144s 17ms/step - loss: 0.5554 - mae: 0.4514 - val_loss: 3.1079 - val_mae: 1.3600 - learning_rate: 0.0010
Epoch 8/50
8002/8002 ━━━━━━━━━━━━━━━━━━━━ 139s 17ms/step - loss: 0.5327 - mae: 0.4427 - val_loss: 2.4510 - val_mae: 1.3468 - learning_rate: 0.0010
Epoch 9/50
8002/8002 ━━━━━━━━━━━━━━━━━━━━ 133s 17ms/step - loss: 0.5194 - mae: 0.4367 - val_loss: 4.3670 - val_mae: 1.7334 - learning_rate: 0.0010
Epoch 10/50
7999/8002 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.5045 - mae: 0.4282
Epoch 10: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
8002/8002 ━━━━━━━━━━━━━━━━━━━━ 143s 17ms/step - loss: 0.5045 - mae: 0.4282 - val_loss: 4.2547 - val_mae: 1.7700 - 

8002/8002 ━━━━━━━━━━━━━━━━━━━━ 149s 18ms/step - loss: 0.4547 - mae: 0.4038 - val_loss: 1.4207 - val_mae: 0.9934 - learning_rate: 5.0000e-04
Epoch 13/50
8002/8002 ━━━━━━━━━━━━━━━━━━━━ 226s 21ms/step - loss: 0.4443 - mae: 0.3987 - val_loss: 1.4243 - val_mae: 0.9718 - learning_rate: 5.0000e-04
Epoch 14/50
8002/8002 ━━━━━━━━━━━━━━━━━━━━ 157s 20ms/step - loss: 0.4390 - mae: 0.3957 - val_loss: 3.1493 - val_mae: 1.5375 - learning_rate: 5.0000e-04
Epoch 15/50
8002/8002 ━━━━━━━━━━━━━━━━━━━━ 162s 20ms/step - loss: 0.4322 - mae: 0.3933 - val_loss: 8.1655 - val_mae: 2.3457 - learning_rate: 5.0000e-04
Epoch 16/50
8002/8002 ━━━━━━━━━━━━━━━━━━━━ 195s 19ms/step - loss: 0.4304 - mae: 0.3921 - val_loss: 5.5301 - val_mae: 1.9862 - learning_rate: 5.0000e-04
Epoch 17/50
8001/8002 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.4254 - mae: 0.3895
Epoch 17: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
8002/8002 ━━━━━━━━━━━━━━━━━━━━ 208s 20ms/step - loss: 0.4254 - mae: 0.3895 - val_loss: 11.


✅ Training completed successfully!
📊 Final Results:
   • Test MAE: 22023.43
   • Test WMAE: 22023.43
   • Test WMAE (Holiday weighted): 21135.60
🔗 View results at: https://wandb.ai/agasi22-free-university-of-tbilisi-/Store-Sales-Forecasting/runs/tijqym0b


best_val_loss,▁
epoch/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
epoch/learning_rate,█████████▄▄▄▄▄▄▄▂▂▂▂▂▁
epoch/loss,█▇▇▆▅▄▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁
epoch/mae,█▇▇▆▅▄▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁
epoch/val_loss,▃▄▅▂▁▁▁▁▂▂▂▁▁▁▃▂▄▃▁▇█▅
epoch/val_mae,▄▅▅▃▁▁▂▂▃▃▂▁▁▂▄▃▅▄▁▇█▆
feature_dimensions,▁
final_train_loss,▁
final_train_mae,▁
final_val_loss,▁


🎉 WandB logging completed!
